# Data Preparation Phase
**Author:** Gabriel Lorenzo I. Santos (gsantos@ateneo.edu)

--------------------

The MIT License (MIT)

Copyright (c) 2020 Gabriel Lorenzo I. Santos

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

-------------------

#### Description
Most data mining activities require data preparation before analysis is undertaken.  According to the CrowdFlower Data Science Report of 2016, 3 out of 5 data scientists spend most of their time in cleaning and organizing data.  **Data Preparation** is the process of selecting, cleaning, constructing, integrating, and formatting data so that it can be used for modeling and analysis. The output of this phase is an _Analytical Data Set_ , which is used in the Modeling phase.

This process usually involves domain knowledge as they define the usual parameters needed for a specific goal.  In this activity, we will implement commonly used data preparation techniques presented by Hadley Wickham in his paper [*Tidy Data*](https://vita.had.co.nz/papers/tidy-data.pdf).

Wickham's paper presented common causes of messiness in data and how to *tidy* them. As his techniques are implemented in R, we will do the same techniques in Python using Pandas functions.

#### What is Tidy Data? (Wickham, 2014)
 - A step along the road to clean data
 - Data that is easy to model, visualize, and aggregate (understandable by computers)
 - Characteristics:
     - 1 variable per column
     - 1 observation/entity per row
     - 1 value belongs to a variable and an observation (1 value per cell)

For now, let's import the following libraries:

In [1]:
import pandas as pd
import numpy as np

In this example, we rely on datasets published by Wickham (2014).

Let's take a look at our sample dataset:

In [2]:
df_preg = pd.read_csv('preg.csv')
df_preg

,sex,pregnant,not_pregnant
0,male,0,5
1,female,1,4


**Question:** What are the variables in this data set?

**Answer:** Sex, Pregnant Identifier, Count

To transform this dataset into its tidy data form, we need to place the $Pregnant Identifier$ as one of its columns.  

In a more technical term, we need to _unpivot_ this data frame.  In Pandas, we use $melt()$ function to do the unpivotting. 

In [3]:
df_preg_unpivot = df_preg.melt(id_vars=["sex"], var_name="is_pregnant", value_name="count")
df_preg_unpivot

,sex,is_pregnant,count
0,male,pregnant,0
1,female,pregnant,1
2,male,not_pregnant,5
3,female,not_pregnant,4


$df\_preg\_unpivot$ complies with the characteristics of a tidy data.  This can now be used for further analysis and other data mining techniques.

Aside from the tidy data properties observed in $df\_preg\_unpivot$, we can also observe the following:
 - No duplicate dimensions with varying measures.
 - Each row is at the most granular level. 
 
_Remember data warehousing?_ This emphasizes the importance of having a data warehouse (and to an extent, data lakes) as the single source of trusted data in an organization.  However, it is understood that not all data is captured or owned by an organization; hence, data scientists in corporations are heavily involved in data preparation or wrangling as they capture data of varying formats from various sources (external sources) and merge them with the structured data from their data warehouse (internal sources).
 
There is a common link between the a data warehouse schema (for RDBMS) or cube (for OLAP systems) and the tidy data: _granularity_ . From the data warehousing modules, we learned that each row in fact table must be an instance of a distinct observation.  In tidy data, each row has to be a distict observation.  

We can then change the way how we think of tidy data, in case you forget its characteristics:
_**What is the entity that we are targetting to analyze?**_

_Entity_ may be a customer, a product, or a store, etc., and is usually identified by a unique identifier.  By identifying the target entity to be analyzed, we also define the granularity of the data set. 

#### Common Causes of Messiness 
Wickham enumarated 5 scenarios where data can be considered as messy data:
1. Column headers are values, not variables names.
2. Multiple variables are stored in one column.
3. Variables are stored in both rows and columns.
4. Multiple types of experimental unit stored in the same table.
5. One type of experimental unit stored in multiple tables.

In this notebook, we will look at the first two types of messy data and how to tidy them.

##### Mess # 1: Column headers are values, not variable names.

In [4]:
df_mess1 = pd.read_csv('mess1.csv')
df_mess1

,religion,less10k,10to20k,20to30k,30to40k,40to50k,50to75k
0,Agnostic,27,34,60,81,76,137
1,Atheist,12,27,37,52,35,70
2,Buddhist,27,21,30,34,33,58
3,Catholic,418,617,732,670,638,1116
4,Refused,15,14,15,11,10,35
5,Evangelical Protestant,575,869,1064,982,881,1486
6,Hindu,1,9,7,9,11,34
7,Historically Black Protestant,228,244,236,238,197,223
8,Jehova's Witness,20,27,24,24,21,30
9,Jewish,19,19,25,25,30,95


There are 3 variables in this data: **religion, income, count**

Similar to $df\_preg$, we need to unpivot the income-labeled columns and place them in 1 column.

In the original R package Wickham developed, this process is called *gather* - combining multiple columns into a single column with a key-value pair format.  

In Pandas, we use the $melt(id\_vars = ["religion"],  var\_name =  "income",  value\_name = "count")$:
 - id_vars = \["religion"\] since every row is identified by religion
 - var_name = "income" since the variables at the column headers are income brackets
 - value_name = "count" since we are taking the frequency or count of households per religion per income bracket

In [5]:
df_tidy1 = df_mess1.melt(id_vars=["religion"], var_name="income", value_name="count")
df_tidy1.head()

,religion,income,count
0,Agnostic,less10k,27
1,Atheist,less10k,12
2,Buddhist,less10k,27
3,Catholic,less10k,418
4,Refused,less10k,15


##### Mess # 2: Multiple Variables are stored in one column.

In [6]:
df_mess2 = pd.read_csv('mess2.csv')
df_mess2

,iso2,year,new_sp,new_sp_m04,new_sp_m514,new_sp_m014,new_sp_m1524,new_sp_m2534,new_sp_m3544,new_sp_m4554,...,new_sp_f04,new_sp_f514,new_sp_f014,new_sp_f1524,new_sp_f2534,new_sp_f3544,new_sp_f4554,new_sp_f5564,new_sp_f65,new_sp_fu
0,AD,1989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AD,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AD,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AD,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AD,1993,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5764,ZW,2004,14581.0,NaN,NaN,187.0,833.0,2908.0,2298.0,1056.0,...,NaN,NaN,225.0,1140.0,2858.0,1565.0,622.0,214.0,111.0,NaN
5765,ZW,2005,13155.0,NaN,NaN,210.0,837.0,2264.0,1855.0,762.0,...,NaN,NaN,269.0,1136.0,2242.0,1255.0,578.0,193.0,603.0,NaN
5766,ZW,2006,12718.0,NaN,NaN,215.0,736.0,2391.0,1939.0,896.0,...,NaN,NaN,237.0,1020.0,2424.0,1355.0,632.0,230.0,96.0,NaN
5767,ZW,2007,10583.0,6.0,132.0,138.0,500.0,3693.0,0.0,716.0,...,7.0,178.0,185.0,739.0,3311.0,0.0,553.0,213.0,90.0,NaN


The data needs to be cleansed due to NaNs. Let's drop them and replace to zeros.

In [7]:
df_mess2.fillna(0, inplace=True)
df_mess2

,iso2,year,new_sp,new_sp_m04,new_sp_m514,new_sp_m014,new_sp_m1524,new_sp_m2534,new_sp_m3544,new_sp_m4554,...,new_sp_f04,new_sp_f514,new_sp_f014,new_sp_f1524,new_sp_f2534,new_sp_f3544,new_sp_f4554,new_sp_f5564,new_sp_f65,new_sp_fu
0,AD,1989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AD,1990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AD,1991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AD,1992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AD,1993,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5764,ZW,2004,14581.0,0.0,0.0,187.0,833.0,2908.0,2298.0,1056.0,...,0.0,0.0,225.0,1140.0,2858.0,1565.0,622.0,214.0,111.0,0.0
5765,ZW,2005,13155.0,0.0,0.0,210.0,837.0,2264.0,1855.0,762.0,...,0.0,0.0,269.0,1136.0,2242.0,1255.0,578.0,193.0,603.0,0.0
5766,ZW,2006,12718.0,0.0,0.0,215.0,736.0,2391.0,1939.0,896.0,...,0.0,0.0,237.0,1020.0,2424.0,1355.0,632.0,230.0,96.0,0.0
5767,ZW,2007,10583.0,6.0,132.0,138.0,500.0,3693.0,0.0,716.0,...,7.0,178.0,185.0,739.0,3311.0,0.0,553.0,213.0,90.0,0.0


Let's have a look at our columns.

In [8]:
df_mess2.columns

Index(['iso2', 'year', 'new_sp', 'new_sp_m04', 'new_sp_m514', 'new_sp_m014',
       'new_sp_m1524', 'new_sp_m2534', 'new_sp_m3544', 'new_sp_m4554',
       'new_sp_m5564', 'new_sp_m65', 'new_sp_mu', 'new_sp_f04', 'new_sp_f514',
       'new_sp_f014', 'new_sp_f1524', 'new_sp_f2534', 'new_sp_f3544',
       'new_sp_f4554', 'new_sp_f5564', 'new_sp_f65', 'new_sp_fu'],
      dtype='object')

This dataset comes from the World Health Organization, and records the counts of confirmed tuberculosis cases by country (iso2), year (year), and demographic group (m04, m514, f04, f514, etc.). 

Looking at our list of columns, *how many total variables do we have?* **4 (country, year, sex, age range)**

Now, how do we approach this? Before we transform the table into its tidy version, let's have another round of data understanding. We can see that there is a column named 'new_sp' which has no demographic information.  Our assumption is that this is the totals column. Totals can be done on the fly but from granularity perspective, we only need 1 type of granularity and that includes demographics sex and age range.

In [9]:
df_mess2.drop(columns=["new_sp"], inplace=True)
df_mess2.head()

,iso2,year,new_sp_m04,new_sp_m514,new_sp_m014,new_sp_m1524,new_sp_m2534,new_sp_m3544,new_sp_m4554,new_sp_m5564,...,new_sp_f04,new_sp_f514,new_sp_f014,new_sp_f1524,new_sp_f2534,new_sp_f3544,new_sp_f4554,new_sp_f5564,new_sp_f65,new_sp_fu
0,AD,1989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AD,1990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AD,1991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AD,1992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AD,1993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we can further proceed in tidying this dataset. We need to melt the table to bring down the demographics variable into one column.

In [10]:
df_mess2 = df_mess2.melt(id_vars=["iso2", "year"], var_name="demog", value_name="cases")
df_mess2.head()

,iso2,year,demog,cases
0,AD,1989,new_sp_m04,0.0
1,AD,1990,new_sp_m04,0.0
2,AD,1991,new_sp_m04,0.0
3,AD,1992,new_sp_m04,0.0
4,AD,1993,new_sp_m04,0.0


Let's clean the elements under the $demog$ column.

In [11]:
# From https://stackoverflow.com/questions/13682044/remove-unwanted-parts-from-strings-in-a-column
df_mess2["demog"] = df_mess2["demog"].map(lambda x: x.lstrip('new_sp_'))
df_mess2

,iso2,year,demog,cases
0,AD,1989,m04,0.0
1,AD,1990,m04,0.0
2,AD,1991,m04,0.0
3,AD,1992,m04,0.0
4,AD,1993,m04,0.0
...,...,...,...,...
115375,ZW,2004,fu,0.0
115376,ZW,2005,fu,0.0
115377,ZW,2006,fu,0.0
115378,ZW,2007,fu,0.0


How do we execute the split of the two variables? We know that the demography data is comprised of two parts: sex (1st letter) and age range (rest of string). Let's split this by getting the substrings and then drop the $demog$ column.

In [12]:
df_mess2["sex"] = df_mess2.demog.str[0]
df_mess2["age"] = df_mess2.demog.str[1:len(df_mess2.demog)]
df_mess2.drop(columns=["demog"], inplace=True)
df_mess2.head()

,iso2,year,cases,sex,age
0,AD,1989,0.0,m,04
1,AD,1990,0.0,m,04
2,AD,1991,0.0,m,04
3,AD,1992,0.0,m,04
4,AD,1993,0.0,m,04


To clean the age column, let's look at the values under it.

In [13]:
df_mess2.age.unique()

array(['04', '514', '014', '1524', '2534', '3544', '4554', '5564', '65',
       'u'], dtype=object)

We can see that there are age groups 0-4, 5-14, and 0-14.  To have a better approach here, a deeper look at the data must be done to see if the age groups 0-4 and 5-14 are relevant or not.   Since the purpose of this notebook is to teach how to clean messy datasets, let's drop the rows consist of 0-4 and 5-14, for now.

In [14]:
df_mess2 = df_mess2[(df_mess2["age"] != "04") & (df_mess2["age"] != "514")]
df_mess2.head()

,iso2,year,cases,sex,age
11538,AD,1989,0.0,m,014
11539,AD,1990,0.0,m,014
11540,AD,1991,0.0,m,014
11541,AD,1992,0.0,m,014
11542,AD,1993,0.0,m,014


Cleanse the data further to come up with our tidy version.

In [15]:
# Replacing the age group data from "014" to "0-14"
df_mess2.replace(["014", "1524", "2534", "3544", "4544", "5564", "65", "u"], ["0-14", "15-24", "25-34", "35-44", "45-44", "55-64", "65+", "unknown"], inplace=True)

# Renaming columns
df_mess2.columns = ["country", "year", "cases", "sex", "age"]

# Rearranging columns and resetting the index
df_tidy2 = df_mess2[["country", "year", "sex", "age", "cases"]].reset_index()
df_tidy2.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,index,country,year,sex,age,cases
0,11538,AD,1989,m,0-14,0.0
1,11539,AD,1990,m,0-14,0.0
2,11540,AD,1991,m,0-14,0.0
3,11541,AD,1992,m,0-14,0.0
4,11542,AD,1993,m,0-14,0.0


------------

#### Exercise
##### Mess # 3: Variables are stored in both rows and columns.

From the 2 previous examples of messy data, we dealt with variables stored in columns.  As an exercise, you will be doing the tidying of our third dataset, which includes variables stored in rows. Don't worry! If this is your first time dealing with messy data, it may look daunting. But in reality, this is still a simple scenario! *(Although according to Wickham, this is the most complicated form of messy data...you'll be the judge :D)*

Let me help you on assessing the data set and I'll let you do the rest.


In [16]:
df_mess3 = pd.read_csv('mess3.csv')
df_mess3

,id,year,month,element,d1,d2,d3,d4,d5,d6,d7,d8
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,NaN,NaN
5,MX17004,2010,3,tmin,NaN,NaN,NaN,NaN,14.2,NaN,NaN,NaN
6,MX17004,2010,4,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MX17004,2010,4,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MX17004,2010,5,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,MX17004,2010,5,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The data comes from the Global Historical Climatology Network for one weather station (MX17004) in Mexico for five months. It has the minimum (tmin) and maximum (tmax) temperature spread out across columns (days). 

In variables stored in rows, the reverse of "unpivot" has to be performed.  In this case, explore the $pivot()$ function in pandas.

Your task is to tidy the data above.  Do the necessary cleanup if needed.  Write your code below and upload both the code and the output file in our Moodle page.  See you on Saturday for the open consultation session!

In [17]:
# Fill up the following information by replacing XXXXXX
student_id = "XXXXXX"
nickname = "XXXXXX"
lastname = "XXXXXX"

# If the data needs to be cleansed, write the code below
# ------------------------------------------------------










# Write your script here for transforming the data set into its tidy version.
# Name your final dataframe as df_tidy3.
# --------------------------------------------------------------------------










# DO NOT TOUCH THE CODE BELOW WHEN TESTING YOUR SCRIPTS
# Uncomment the code below only when you are done with the script to export the table for submission.
# ------------------------------------------------------------------------------------

#filename = student_id + "_" + lastname + "_" + nickname + ".csv"
#df_tidy3.to_csv(filename)

------------------

##### Bonus Exercise
Wickham also mentioned about the 4th and 5th types of messy data.  Included in this repository is mess45.csv.  Explore the dataset and see if you can devise a strategy on how to deal with such datasets.  More information about Wickham's approach in https://vita.had.co.nz/papers/tidy-data.pdf .

------------------

#### References
Hadley Wickham. 2014. Tidy Data. Journal of Statistical Software 59, 10 (2014). DOI:http://dx.doi.org/10.18637/jss.v059.i10

Stuart Clarke. 2017. Data Manipulation. Video. In *Getting Started with Data Science*. openSAP. https://open.sap.com/courses/ds1/items/1VOrErA8HpbU7DMC0FZRWl